# Canonical Objects

In [3]:
import networkx as nx
from itertools import combinations, permutations
from copy import deepcopy
import pickle
from tqdm.notebook import tqdm
from scipy.special import comb
import matplotlib.pyplot as plt
from pref_voting.generate_weighted_majority_graphs import *

In [4]:
def display_wwt(wwt, edge_attr='weight'):
    """Display a weighted (weak) tournament with edge weights"""
    pos = nx.circular_layout(wwt)
    nx.draw(wwt, pos, font_size=20, node_color='blue', font_color='white',
            node_size=700, width=1, with_labels=True)
    
    edge_labels = nx.get_edge_attributes(wwt, edge_attr)
    nx.draw_networkx_edge_labels(wwt, pos, edge_labels=edge_labels, label_pos=0.3)
    plt.show()

def findsubsets(s, n):
    """all subsets of the list s of size n""" 
    return [set(i) for i in combinations(s, n)] 

def is_maj_preferred(wt, c1, c2): 
    """True if c1 is majority preferred to c2"""
    return wt.has_edge(c1, c2)

def is_tournament(wt):
    """test if a weak tournament is a tournament"""
    candidates = wt.nodes
    is_t = True
    for c1 in candidates: 
        for c2 in candidates: 
            if c1 != c2 and not is_maj_preferred(wt,c1,c2) and not is_maj_preferred(wt,c2,c1):
                is_t = False
    return is_t

## Canonical Weak Tournaments

The following code generates a representative from each isomorhpism class of weak tournaments. 

We call these representatives the **canonical weak tournaments**.

In [5]:
%%time 

# Warning: Constructing the canonical weak tournaments with 7 candidates takes a long time!

numbers_of_candidates = [2, 3, 4, 5, 6]#, 7]

#  nx.graph_atlas_g() is a list of all the undirected graphs with up to 7 nodes
# See https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.generators.atlas.graph_atlas_g.html

undirected_graphs_size_2 = nx.graph_atlas_g()[2:4] # undirected graphs with 3 nodes
undirected_graphs_size_3 = nx.graph_atlas_g()[4:8] # undirected graphs with 3 nodes
undirected_graphs_size_4 = nx.graph_atlas_g()[8:19] # undirected graphs with 4 nodes
undirected_graphs_size_5 = nx.graph_atlas_g()[19:53] # undirected graphs with 5 nodes
undirected_graphs_size_6 = nx.graph_atlas_g()[53:209] # undirected graphs with 6 nodes
#undirected_graphs_size_7 = nx.graph_atlas_g()[209::] # undirected graphs with 7 nodes

undirected_graphs = {
    2: undirected_graphs_size_2, 
    3: undirected_graphs_size_3, 
    4: undirected_graphs_size_4,
    5: undirected_graphs_size_5, 
    6: undirected_graphs_size_6,
    #7: undirected_graphs_size_7
}

canonical_weak_tournaments = {nc: list() for nc in numbers_of_candidates}

for nc in numbers_of_candidates: 
    
    for g in tqdm(undirected_graphs[nc]):

        directed_graphs_from_g_by_score_seq = dict()
        
        num_edges = len(g.edges)

        # for n=0 to the number of edges
        for n in range(num_edges + 1): 

            # find all the subsets of edges of size n
            forward_edges = findsubsets(g.edges, n)

            # for each set of edges
            for f_edges in forward_edges: 

                # f_edges is the set of forward edges, the other edges in g point in the opposite direction
                d_edges = list(f_edges) + [(e[1], e[0]) for e in g.edges if e not in f_edges]

                # create a directed graph with the nodes from g and directed edges in d_edges
                d_graph = nx.DiGraph()
                d_graph.add_nodes_from(g.nodes)
                d_graph.add_edges_from(d_edges)

                # check if we have already seen a graph isomorphic to d_graph, 
                # using the fact that a graph isomorphic to d_graph must have the same score sequence as d_graph

                d_score_seq = tuple(sorted([d_graph.out_degree(x) - d_graph.in_degree(x) for x in d_graph.nodes]))

                if not d_score_seq in directed_graphs_from_g_by_score_seq.keys():
                    directed_graphs_from_g_by_score_seq[d_score_seq] = [d_graph]
                    canonical_weak_tournaments[nc].append(d_graph)

                else:
                    score_type = directed_graphs_from_g_by_score_seq[d_score_seq]
                    if all([not nx.is_isomorphic(d_graph,_t) for _t in score_type]): 
                        directed_graphs_from_g_by_score_seq[d_score_seq].append(d_graph)
                        canonical_weak_tournaments[nc].append(d_graph)
                    
    # save the weak tournaments
    pickle.dump(canonical_weak_tournaments[nc], open(f"weak_tourns/weak_tourns_{nc}.pkl", "wb"))

print("Done creating canonical weak tournaments.\n")

for nc in numbers_of_candidates: 
    print(f"There are {len(canonical_weak_tournaments[nc])} canonical weak tournaments for {nc} candidates")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/156 [00:00<?, ?it/s]

In [ ]:
numbers_of_candidates = [2, 3, 4, 5, 6]#, 7]

# Set aside the tournaments
canonical_tournaments = {n_cands: list() for n_cands in canonical_weak_tournaments.keys()}
for n_cands in canonical_weak_tournaments.keys():
    for t in canonical_weak_tournaments[n_cands]:
        if is_tournament(t):
            canonical_tournaments[n_cands].append(t)

# save the tournaments
for nc in canonical_tournaments.keys():
    pickle.dump(canonical_tournaments[nc], open(f"tourns/tourns_{nc}.pkl", "wb"))

print("Done creating canonical tournaments.\n")

for nc in numbers_of_candidates: 
    print(f"There are {len(canonical_tournaments[nc])} canonical tournaments for {nc} candidates")

Done creating canonical tournaments.

There are 1 canonical tournaments for 2 candidates
There are 2 canonical tournaments for 3 candidates
There are 4 canonical tournaments for 4 candidates
There are 12 canonical tournaments for 5 candidates
There are 56 canonical tournaments for 6 candidates


## Canonical Weighted Tournaments

The following code generates a representative from each isomorhpism class of uniquely weighted tournaments with weights from $\{2, 4, 6, 8, 10, 12\}$. 

We call these representatives the **canonical weighted tournaments**.

In [ ]:
%%time 

all_num_candidates = [2, 3, 4]

weight_domain = [2, 4, 6, 8, 10, 12]

canonical_weighted_tournaments = {nc: None for nc in all_num_candidates}

for num_cands in tqdm(all_num_candidates): 
    canonical_weighted_tournaments[num_cands] = [mg.to_networkx() for mg in enumerate_uniquely_weighted_margin_graphs(num_cands, weight_domain)]

# save the weighted tournaments
for nc in canonical_weighted_tournaments.keys():
    pickle.dump(canonical_weighted_tournaments[nc], open(f"weighted_tourns/weighted_tourns_{nc}.pkl", "wb"))

for nc in all_num_candidates: 
    print(f"There are {len(canonical_weighted_tournaments[nc])} canonical weighted tournaments for {nc} candidates")

  0%|          | 0/3 [00:00<?, ?it/s]

There are 6 canonical weighted tournaments for 2 candidates
There are 160 canonical weighted tournaments for 3 candidates
There are 1920 canonical weighted tournaments for 4 candidates
CPU times: user 169 ms, sys: 15.6 ms, total: 185 ms
Wall time: 186 ms
